# Stochastic Gradient Descent

## Introduction
<br>

모든 기계 학습 작업과 마찬가지로 학습 데이터 세트로 시작합니다. 훈련 데이터의 각 예는 예상 타겟 (출력)와 함께 일부 피처 (입력)로 구성됩니다. 네트워크 훈련은 피처을 타겟으로 변환할 수 있도록 가중치를 조정하는 것을 의미합니다. 
훈련 데이터 외에도 다음 두 가지가 더 필요합니다.

* 네트워크의 예측이 얼마나 좋은지 측정하는 loss function
* 가중치를 변경하는 방법을 네트워크에 알려줄 수 있는 optimizer

## The Loss Function
<br>

**loss function**는 대상의 실제 값과 모델이 예측하는 값 간의 차이를 측정합니다.

regression 문제에 대한 일반적인 손실 함수는 **평균 절대 오차(mean absolute error)** 또는 **MAE** 입니다. 각 예측 y_pred에 대해 MAE는 절대 차이 `abs(y_true - y_pred)`에 의해 실제 목표 `y_true`와의 불일치를 측정합니다.

데이터 세트의 총 MAE 손실은 이러한 모든 절대값 차이의 평균입니다.

<img src="./images/dl09.png" width=500>

> 평균 절대 오차는 적합 곡선과 데이터 점 사이의 평균 길이입니다.


## The Optimizer - Stochastic Gradient Descent
<br>

**Optimizer**는 손실을 최소화하기 위해 가중치를 조정하는 알고리즘입니다.

딥 러닝에 사용되는 거의 모든 최적화 알고리즘은 확률적 경사 하강법(stochastic gradient descent) 입니다. 네트워크를 단계적으로 훈련시키는 반복 알고리즘입니다. 훈련의 한 단계는 다음과 같습니다.

1. 훈련 데이터를 샘플링하고 네트워크를 통해 실행하여 예측합니다.
2. 예측과 실제 값 사이의 손실을 측정합니다.
3. 마지막으로 손실을 줄이는 방향으로 가중치를 조정합니다.

그런 다음 손실이 원하는 만큼 작아질 때까지 (또는 더 이상 감소하지 않을 때까지) 계속 반복합니다.

<img src="./images/dl10.gif">

### Learning Rate and Batch Size
<br>

선은 각 배치의 방향으로 약간만 이동합니다 (끝까지 이동하는 대신). 이러한 변화의 크기는 **학습률(learning rate)**에 의해 결정됩니다. 학습률이 낮을수록 네트워크는 가중치가 최상의 값으로 수렴되기 전에 더 많은 미니 배치를 확인해야 합니다.

### Adding the Loss and Optimizer
<br>

모델을 정의한 후 모델의 compile 메서드를 사용하여 loss function과 optimizer를 추가할 수 있습니다.

```python
model.compile(
    optimizer="adam",
    loss="mae",
)
```

## Example - Red Wine Quality
<br>

실습을 위해 [Red Wine Quality](https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009) 데이터 세트를 사용합니다.

이 데이터 세트는 약 1600 개의 포르투갈 적포도주에 대한 물리 화학적 측정 값으로 구성됩니다. 또한 블라인드 맛 테스트의 각 와인에 대한 품질 등급이 포함되어 있습니다. 

In [ ]:
import pandas as pd
from IPython.display import display

red_wine = pd.read_csv('../input/dl-course-data/red-wine.csv')

In [ ]:
# Create training and validation splits
red_wine.head()

In [ ]:
df_train = red_wine.sample(frac=0.7, random_state=0)
df_valid = red_wine.drop(df_train.index)

In [ ]:
df_train.shape, df_valid.shape

In [ ]:
df_train.head(10)

In [ ]:
max_ = df_train.max(axis=0)
min_ = df_train.min(axis=0)

In [ ]:
df_train = (df_train - min_ ) / (max_ - min_)

In [ ]:
df_valid = (df_valid - min_ ) / (max_ - min_)

In [ ]:
df_train.min(), df_train.max()

In [ ]:
X_train = df_train.drop('quality', axis=1)
Y_train = df_train['quality']

X_valid = df_valid.drop('quality',axis=1)
Y_valid = df_valid['quality']

In [ ]:
X_train.head()

In [ ]:
Y_train.head()

In [ ]:
# Scale to [0, 1]


In [ ]:
# Split features and target


여기서는 1500 개 이상의 뉴런이 있는 3 레이어 네트워크를 선택했습니다. 이 네트워크는 데이터에서 상당히 복잡한 관계를 학습할 수 있습니다.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    # write your code here.
    layers.Dense(512, activation = 'relu', input_shape=[11]),
    layers.Dense(512, activation = 'relu'),
    layers.Dense(512, activation = 'relu'),
    layers.Dense(1),
])

In [ ]:
model.summary()

모델을 정의한 후 optimizer 와 loss function과 함께 컴파일합니다.

In [ ]:
model.compile(
    optimizer = 'adam' ,
    loss = 'mae',
)

Keras에게 한 번에 256 행의 훈련 데이터 (batch_size)를 optimizer에 공급하고 dataset (the epochs) 전체에 걸쳐 10 번 수행하도록 지시합니다.

In [ ]:
X_train.shape[0] / 256 * 10

In [ ]:
history = model.fit(
    # write your code here.
    X_train, Y_train,
    validation_data=(X_valid, Y_valid),
    batch_size=256,
    epochs=10
)

손실을 보는 더 좋은 방법은 그래프를 그리는 것입니다. 실제로 fit 메서드는 History 객체에서 훈련 중에 발생한 손실 기록을 유지합니다. 데이터를 Pandas 데이터 프레임으로 변환하여 쉽게 플로팅 할 수 있습니다.

In [ ]:
import pandas as pd

In [ ]:
history_df = pd.DataFrame(history.history)
history_df['loss'].plot(grid=True);

In [ ]:
history_df['val_loss'].plot(grid=True);